# PyROOT Cut Efficiency
Sebastiaan Venendaal<br>
<ul>
    <li>Created on 01-11-20</li>
    <li>Last edited on 01-11-20</li>
</ul>

In [69]:
# Imports
import numpy as np
import ROOT
import pandas as pd

In [3]:
# Canvas config
%jsroot on
c_i = 0

## Data

In [4]:
# Include header file
ROOT.gInterpreter.Declare('#include "../Ntp.h"')

# Make a Tree Chain
chain = ROOT.TChain('Simp')
# Fill Chain with all available Runs
chain.Add('../../Simp.root')
    
# Print number of entries
chain_events = chain.GetEntries()
print('Data Entries in TChain:', chain_events)

# Visual check
#chain.Print()
    

Data Entries in TChain: 889410


## Cuts

In [83]:
# Cut configuration
lay_crit = '>=' # criterion symbol for bad charge status in tracker layers
geo_crit = '1.2' # criterion strength of geo-magnetic cut-off (standard convention is 1.2)

# CUTS DataFrame
cut_df = pd.DataFrame(columns=['Name', 'Cut', 'Instrument', 'Description'])
# Append all single CUTS
cut_df = cut_df.append({'Name': 'Crig', 'Cut': ROOT.TCut('trk_rig > 0 && trk_rig <= 22'),
          'Instrument': 'General Cut', 'Description': 'Deuteron identification range'}, ignore_index=True)
cut_df = cut_df.append({'Name': 'Cpar', 'Cut': ROOT.TCut('status % 10 == 1'),
          'Instrument': 'Combi', 'Description': 'Single particle events'}, ignore_index=True)
cut_df = cut_df.append({'Name': 'Ccon', 'Cut': ROOT.TCut('abs(tof_beta-rich_beta)/tof_beta < 0.05'),
          'Instrument': 'TOF', 'Description': 'Consistancy between TOF and RICH beta measurement'}, ignore_index=True)
cut_df = cut_df.append({'Name': 'Cbet', 'Cut': ROOT.TCut('tof_beta > 0'),
          'Instrument': 'TOF', 'Description': 'Down-going particles'}, ignore_index=True)
cut_df = cut_df.append({'Name': 'Cchi', 'Cut': ROOT.TCut('trk_chisqn[0] < 10 && trk_chisqn[1] < 10 && trk_chisqn[0] > 0 && trk_chisqn[1] > 0'),
          'Instrument': 'Tracker', 'Description': 'Well-reconstructed particle tracks'}, ignore_index=True)
cut_df = cut_df.append({'Name': 'Cinn', 'Cut': ROOT.TCut('trk_q_inn > 0.80 && trk_q_inn < 1.30'),
          'Instrument': 'Tracker', 'Description': 'Single charge particles'}, ignore_index=True)
cut_df = cut_df.append({'Name': 'Clay', 'Cut': ROOT.TCut('trk_q_lay[4]{0}0 && trk_q_lay[1]{0}0 && trk_q_lay[2]{0}0 && trk_q_lay[3]{0}0 &&'.format(lay_crit) +
                                     'trk_q_lay[5]{0}0 && trk_q_lay[6]{0}0 && trk_q_lay[7]{0}0 && trk_q_lay[8]{0}0 &&'.format(lay_crit) +
                                     'trk_q_lay[0]{0}0'.format(lay_crit)),
          'Instrument': 'Tracker', 'Description': 'No bad charge status throughout tracker'}, ignore_index=True)
cut_df = cut_df.append({'Name': 'Cgeo', 'Cut': ROOT.TCut('trk_rig > {0}*cf'.format(geo_crit)),
          'Instrument': 'Tracker', 'Description': 'Geo-magnetic cut-off'}, ignore_index=True)
# Print CUTS for visual check
print(cut_df)

   Name                                                Cut   Instrument  \
0  Crig                       trk_rig > 0 && trk_rig <= 22  General Cut   
1  Cpar                                   status % 10 == 1        Combi   
2  Ccon            abs(tof_beta-rich_beta)/tof_beta < 0.05          TOF   
3  Cbet                                       tof_beta > 0          TOF   
4  Cchi  trk_chisqn[0] < 10 && trk_chisqn[1] < 10 && tr...      Tracker   
5  Cinn               trk_q_inn > 0.80 && trk_q_inn < 1.30      Tracker   
6  Clay  trk_q_lay[4]>=0 && trk_q_lay[1]>=0 && trk_q_la...      Tracker   
7  Cgeo                                   trk_rig > 1.2*cf      Tracker   

                                         Description  
0                      Deuteron identification range  
1                             Single particle events  
2  Consistancy between TOF and RICH beta measurement  
3                               Down-going particles  
4                 Well-reconstructed particle tra

In [108]:
cut_df[cut_df['Name'] == 'Cchi']['Cut'].values[0]

In [115]:
current_cut = cut_df[cut_df['Name'] == 'Cchi']['Cut'].values[0]
current_inst = cut_df[cut_df['Name'] ==  'Cchi']['Instrument'].values[0]

sub_df = cut_df[(cut_df['Instrument'] != current_inst)]
sub_df = sub_df.append(cut_df[cut_df['Name'] == 'Cchi'])

print(sub_df)

   Name                                                Cut   Instrument  \
0  Crig                       trk_rig > 0 && trk_rig <= 22  General Cut   
1  Cpar                                   status % 10 == 1        Combi   
2  Ccon            abs(tof_beta-rich_beta)/tof_beta < 0.05          TOF   
3  Cbet                                       tof_beta > 0          TOF   
4  Cchi  trk_chisqn[0] < 10 && trk_chisqn[1] < 10 && tr...      Tracker   

                                         Description  
0                      Deuteron identification range  
1                             Single particle events  
2  Consistancy between TOF and RICH beta measurement  
3                               Down-going particles  
4                 Well-reconstructed particle tracks  


In [65]:
# CUT CONFIG
lay_crit = '>=' # criterion symbol for bad charge status in tracker layers
geo_crit = '1.2' # criterion strength of geo-magnetic cut-off (standard convention is 1.2)

# Standard cut in rigidity range
CGb = ROOT.TCut('trk_rig > 0 && trk_rig <= 22') # deuteron identification range
CS = CGb

# Cuts grouped by INSTRUMENT
# Combi
CQb = ROOT.TCut('status % 10 == 1') # single particle events
CCo = CQb
# Tracker
CZa = ROOT.TCut('trk_q_inn > 0.80 && trk_q_inn < 1.30') # single charge particles
CZb = ROOT.TCut('trk_q_lay[4]{0}0 && trk_q_lay[1]{0}0 && trk_q_lay[2]{0}0 && trk_q_lay[3]{0}0 &&'.format(lay_crit) +
                'trk_q_lay[5]{0}0 && trk_q_lay[6]{0}0 && trk_q_lay[7]{0}0 && trk_q_lay[8]{0}0 &&'.format(lay_crit) +
                'trk_q_lay[0]{0}0'.format(lay_crit)) # no bad charge status throughout tracker
CR = ROOT.TCut('trk_rig > {0}*cf'.format(geo_crit)) # geo-magnetic cut-off
CQa = ROOT.TCut('trk_chisqn[0] < 10 && trk_chisqn[1] < 10 && trk_chisqn[0] > 0 && trk_chisqn[1] > 0') # well-reconstructed particles
CTr = CZa + CZb + CR + CQa
# TOF
CGa = ROOT.TCut('tof_beta > 0') # down-going particles
CTo = CGa

CAll = CS + CCo + CTr + CTo

## Histogram Cuts

In [68]:
canvas = ROOT.TCanvas("ratio_example", "ratio_example", 800, 500)
chain.Draw('trk_rig >> hist1(50, 0, 22)'.format(chain.GetEntries()), CAll-CGa)
canvas.Draw()


NotImplementedError: 

Warning in <TCanvas::Constructor>: Deleting canvas with same name: ratio_example


In [46]:
h1 = ROOT.TH1F("h1", "h1", 50, 0, 22)
h2 = ROOT.TH1F("h2", "h2", 50, 0, 22)

for i in range(chain.GetEntries()):
    chain.GetEntry(i)
    if (chain.trk_rig < 22 and chain.trk_rig > 0):
        h2.Fill(chain.trk_rig)
        if chain.tof_beta > 0:
            h1.Fill(chain.trk_rig)
            
h3 = h1.Clone("h3")
h3.Divide(h2)

canvas = ROOT.TCanvas("ratio", "ratio", 800, 500)

#h1.Draw()
#h2.Draw('SAME')
h3.Draw('SAME')
canvas.Draw()
    
    

Warning in <TROOT::Append>: Replacing existing TH1: h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h2 (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: ratio


In [109]:
canvas = ROOT.TCanvas("ratio_example", "ratio_example", 800, 500)
chain.Draw('trk_rig >> hist1(50, 0, 22)'.format(chain.GetEntries()), cut_df[cut_df['Name'] == 'Cchi']['Cut'].values[0])
chain.Draw('trk_rig >> hist2(50, 0, 22)'.format(chain.GetEntries()), '', 'SAME')
h3 = ROOT.hist1.Clone("hist3")
h3.Divide(ROOT.hist2)

# ROOT.hist1.Scale(1/chain_events)
# ROOT.hist2.Scale(1/chain_events*500)

h3.Draw()
canvas.Draw()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: ratio_example


In [ ]:
def cut_plot(par, title='', xlabel='', ylabel='events', stats=False,  bins=200,
             xrange=(0,0), yrange=(0,0), log_y=True, prop=False, lx=0.65, ly=0.6):
    
    # Create Canvas
    global c_i
    canvas = ROOT.TCanvas("{}".format(c_i),"{}".format(c_i), 800, 500)
    # Draw Histogram and store in ROOT.histo
    xrange = str(xrange)[1:-1]
    chain.Draw('{} >> hist1({}, {})'.format(par, bins, xrange), '')
    chain.Draw('{} >> hist2({}, {})'.format(par, bins, xrange), CG, 'SAME')
    chain.Draw('{} >> hist3({}, {})'.format(par, bins, xrange), CG+CQ, 'SAME')
    # Coloring
    ROOT.hist1.SetLineColor(ROOT.kBlue)
    ROOT.hist2.SetLineColor(ROOT.kRed)
    ROOT.hist3.SetLineColor(ROOT.kGreen)
    ROOT.hist4.SetLineColor(ROOT.kOrange)
    ROOT.hist5.SetLineColor(ROOT.kBlack)
    ROOT.hist5.SetLineWidth(2)
    # Labelling
    if title == '':
        title = par
    ROOT.hist1.SetTitle(title)
    ROOT.hist1.GetXaxis().SetTitle(xlabel)
    ROOT.hist1.GetYaxis().SetTitle(ylabel)
    ROOT.hist1.SetStats(stats)
    # Axis
    if yrange != (0,0):
        ROOT.hist1.SetAxisRange(yrange[0], yrange[1], "Y")
    canvas.SetLogy(log_y)
    
    # Legend
    legend = ROOT.TLegend(lx, ly,lx+.20, ly+.15)
    legend.AddEntry(ROOT.hist1, 'None')
    legend.AddEntry(ROOT.hist2, 'CG')
    legend.AddEntry(ROOT.hist3, 'CG+CQ')
    legend.AddEntry(ROOT.hist4, 'CG+CQ+CZ')
    legend.AddEntry(ROOT.hist5, 'CG+CQ+CZ+CR')
    legend.SetLineWidth(0)
    legend.SetTextSize(10)
    
    # Proportion of events
    if prop is True:
        he1 = ROOT.hist1.GetEntries(); print('None: ', he1, '({:.1f} %)'.format(he1/chain_events*100))
        he1 = ROOT.hist2.GetEntries(); print('CG: ', he1, '({:.1f} %)'.format(he1/chain_events*100))
        he1 = ROOT.hist3.GetEntries(); print('CG+CQ: ', he1, '({:.1f} %)'.format(he1/chain_events*100))
        he1 = ROOT.hist4.GetEntries(); print('CG+CQ+CZ: ', he1, '({:.1f} %)'.format(he1/chain_events*100))
        he1 = ROOT.hist5.GetEntries(); print('CG+CQ+CZ+CR: ', he1, '({:.1f} %)'.format(he1/chain_events*100))
        
    # Increment canvas counter
    c_i += 1
    
    return canvas, legend
    

----
END OF NOTEBOOK